In [1]:
import sys
sys.executable

'C:\\Users\\vietx\\anaconda3\\python.exe'

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Khám phá dữ liệu

### Đọc dữ liệu từ file csv


In [3]:
billboards = pd.read_csv('charts.csv')
billboards.head()

,date,rank,song,artist,last-week,peak-rank,weeks-on-board
0,2021-11-06,1,Easy On Me,Adele,1.0,1,3
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14
3,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19
4,2021-11-06,5,Bad Habits,Ed Sheeran,5.0,2,18


In [4]:
billboards = billboards.rename(columns={'last-week': 'last_week',
                                        'peak-rank': 'peak_rank',
                                        'weeks-on-board': 'weeks_on_board'})

### Dữ liệu có bao nhiêu dòng và bao nhiêu cột?

In [5]:
num_rows = len(billboards)
num_cols = len(billboards.columns)

num_rows, num_cols

(330087, 7)

### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?
- Mỗi dòng của dữ liệu là thông tin bài hát và các thông số xếp hạng của bài hát trong bảng xếp hạng trong một tuần
- Dựa theo dữ liệu ta thấy rằng các dòng không có ý nghĩa khác nhau

### Dữ liệu có các dòng bị lặp không?


In [6]:
duplicated = np.sum(billboards.duplicated()) != 0
duplicated

False

### Mỗi cột dữ liệu có ý nghĩa:
- Date: Ngày bảng xếp hạng tuần của Billboard được công bố
- Rank: Thứ hạng của bài hát trên bảng xếp hạng tuần Billboard được công bố vào ngày ở cột đầu tiên.
- Song: Tên bài hát được xếp hạng
- Artist: Ca sỹ hát bài hát
- Last-week: Thứ hạng của bài hát đó trên bảng xếp hạng tuần trước của Billboard.
- Peak-rank: Thứ hạng cao nhất của bài hát đó trên tất cả các bảng xếp hạng tuần của Billboard.
- Weeks-on-board: ...


### Kiểu dữ liệu của mỗi cột

In [7]:
col_dtypes = billboards.dtypes

col_dtypes

date               object
rank                int64
song               object
artist             object
last_week         float64
peak_rank           int64
weeks_on_board      int64
dtype: object

Các cột có kiểu dữ liệu chưa phù hợp:
- Cột date đang ở dạng object, chưa phù hợp để xử lý nên ta cần chuyển về kiểu datetime.
- Cột last-week nên có dữ liệu là int thay vì float64.
- Cột song & artist có kiểu là object nhưng có thể kiểu dữ liệu thật sự là string

# Tiền xử lý

### Chuyển dtype của cột "Date" sang datetime 

In [8]:
billboards['date'] = pd.to_datetime(billboards['date'], format = '%Y-%m-%d')

### Chuyển dtype của cột last-week sang int

In [9]:
billboards['last_week'] = billboards['last_week'].replace(np.nan, 0).astype('int64')

Ta thay thế các ô có giá trị nan bằng 0. Việc thay thế này là hợp lý vì không có thứ hạng 0 trên BXH. Khi đó, ta có thể ngầm hiểu khi cột last-week có giá trị là 0 thì bài hát đó lần đầu lọt vào BXH.

# Khám phá dữ liệu (tiếp tục)

### Với mỗi cột có kiểu dữ liệu dạng số (numerical), các giá trị được phân bố như thế nào?

In [10]:
nume_cols = ['date', 'rank', 'last_week', 'peak_rank', 'weeks_on_board']

def missing_ratio(col):
    return col.isnull().sum() / len(col)

nume_col_profiles_df = billboards.agg(['min', 'max', missing_ratio])[nume_cols]

nume_col_profiles_df

,date,rank,last_week,peak_rank,weeks_on_board
min,1958-08-04 00:00:00,1.0,0.0,1.0,1.0
max,2021-11-06 00:00:00,100.0,100.0,100.0,90.0
missing_ratio,0.0,0.0,0.0,0.0,0.0


Không có gì bất thường với các giá trị min, max, missing_ratio của các cột. Ta có thể kiểm chứng giá trị max của weeks-on-board bằng cách tìm trên google bài hát có mặt trên BXH Billboard 90 tuần. Sau khi tra cứu, ta thấy bài hát đó chính là "Blinding Light" và chính thức rời khỏi BXH vào tháng 9/2021. Khá hợp lý khi BXH của ta có tuần cuối cùng được thống kê vào tháng 11/2021.

### Với mỗi cột có kiểu dữ liệu dạng phân loại (categorical),các giá trị được phân bố như thế nào?

In [11]:
cate_cols = ['artist', 'song']

def num_diff_vals(col):
    return col.nunique()

def diff_vals(col):
    return col[col.isna()==False].unique()

cate_col_profiles_df = billboards.agg([missing_ratio, num_diff_vals, diff_vals])[cate_cols]

cate_col_profiles_df

,artist,song
missing_ratio,0.0,0.0
num_diff_vals,10205,24620
diff_vals,"[Adele, The Kid LAROI & Justin Bieber, Lil Nas...","[Easy On Me, Stay, Industry Baby, Fancy Like, ..."


Ta thấy rằng không có gì quá bất thường. Mising ratio bằng 0 là hợp lý. Mặt khác, một ca sỹ có thể có nhiều bài hát nên việc số lượng bài hát nhiều hơn gấp hơn 2 lần so với số lượng ca sỹ là bình thường.

# Đưa ra câu hỏi

### Thêm ID cho bài hát

Tạo một dataframe mới là danh sách các bài hát, các ca sĩ và ID mà nhóm tự đặt cho từng bài hát

In [12]:
songs = pd.DataFrame(data=billboards, columns=['song', 'artist'])\
            .drop_duplicates()\
            .reset_index(drop=True)\
            .reset_index()\
            .rename(columns={'index': 'ID'})

In [13]:
songs

,ID,song,artist
0,0,Easy On Me,Adele
1,1,Stay,The Kid LAROI & Justin Bieber
2,2,Industry Baby,Lil Nas X & Jack Harlow
3,3,Fancy Like,Walker Hayes
4,4,Bad Habits,Ed Sheeran
...,...,...,...
29676,29676,Stay,The Ames Brothers
29677,29677,Over And Over,Thurston Harris
29678,29678,Little Serenade,The Ames Brothers
29679,29679,I'll Get By (As Long As I Have You),Billy Williams


Thêm cột ID vào dataframe billboards, với ID tương ứng với từng bài hát. Dùng hàm merge trong Pandas để merge 2 dataframe là songs và billboards với điều kiện song và artist giống nhau

In [14]:
new_billboards = pd.merge(songs, billboards, how='inner', on=['song', 'artist'])

### Với mỗi tháng có bao nhiêu bài hát mới chưa xuất hiện trong BXH thuộc các tuần trước đó?

Ý nghĩa: để có thể biết được trong mỗi tháng có bao nhiêu bài hát trụ lại được ở BXH và bao nhiêu bài bứt phá để vào được BXH

Trả lời:
- Ta sẽ lọc ra các dòng mà có last_week=0 (nghĩa là chưa từng xuất hiện trong BXH)
- Sau khi lọc ra thì sẽ groupby theo year và month để đếm số lượng bài hát mới theo ID
- Vì dữ liệu bắt đầu từ tháng 8 năm 1958 và kết thúc vào tháng 11 năm 2021 nên các tháng trước đó của năm 1958 và tháng 12 của năm 2021 sẽ có giá trị là NaN

Thêm cột month và year ứng với cột date vào dataframe

In [15]:
new_billboards['month'] = pd.DatetimeIndex(new_billboards['date']).month
new_billboards['year'] = pd.DatetimeIndex(new_billboards['date']).year

In [16]:
new_billboards.head()

,ID,song,artist,date,rank,last_week,peak_rank,weeks_on_board,month,year
0,0,Easy On Me,Adele,2021-11-06,1,1,1,3,11,2021
1,0,Easy On Me,Adele,2021-10-30,1,68,1,2,10,2021
2,0,Easy On Me,Adele,2021-10-23,68,0,68,1,10,2021
3,1,Stay,The Kid LAROI & Justin Bieber,2021-11-06,2,2,1,16,11,2021
4,1,Stay,The Kid LAROI & Justin Bieber,2021-10-30,2,2,1,15,10,2021


In [17]:
df = new_billboards[new_billboards['last_week']==0].groupby(['year', 'month'])['ID'].count()

years = list(set(new_billboards['year'])) # lấy ra list các năm có trong dataframe
new_songs_in_months = pd.DataFrame(index=np.arange(1,13), columns=years) # tạo một dataframe kết quả với các cột là các năm
                                                                         # các index là các tháng trong năm

new_songs_in_months[years[0]][-5:] = df.values[:5] # vì năm đầu tiên bắt đầu từ tháng 8 nên sẽ gán 5 tháng trong năm đó là
                                                   # 5 giá trị đầu tiên trong df.values
new_songs_in_months[years[1:-1]] = df.values[5:-11].reshape(12,len(years)-2) # reshape df.values thành (12,) để gán vào các
                                                                             # năm ở giữa
new_songs_in_months[years[-1]][:-1] = df.values[-11:] # năm cuối cùng có dữ liệu tới tháng 11 nên sẽ gán bằng 11 giá trị
                                                      # cuối cùng trong df.values

new_songs_in_months

,1958,1959,1960,1961,1962,1963,1964,1965,1966,1967,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
1,NaN,55,36,66,50,50,69,42,55,50,...,70,56,66,56,55,78,50,51,76,116
2,NaN,61,60,75,54,59,71,59,68,59,...,70,49,60,70,54,46,55,62,56,42
3,NaN,78,51,46,74,51,51,68,51,53,...,39,40,44,31,43,48,34,39,53,36
4,NaN,40,40,42,39,51,40,36,46,51,...,36,39,40,48,45,34,41,36,35,60
5,NaN,44,42,40,39,29,42,42,37,47,...,39,39,36,36,45,30,35,37,40,74
6,NaN,31,35,27,30,42,28,40,34,33,...,34,33,33,35,32,43,31,31,29,66
7,NaN,27,31,35,37,29,40,25,32,31,...,22,33,29,22,33,28,25,19,29,76
8,158,27,36,21,31,28,29,33,32,25,...,22,28,32,25,25,32,27,30,35,53
9,70,23,25,23,29,27,22,22,36,27,...,32,28,22,27,40,31,37,24,28,80
10,43,38,31,26,42,33,41,30,34,30,...,55,43,39,42,38,55,63,52,30,77


### Sắp xếp các bài hát có số tuần liên tiếp đạt top 1 theo thứ tự giảm dần

Ý nghĩa: để biết được những bài hát nào có sức ảnh hưởng lớn và liên tục trong một khoảng thời gian dài

Trả lời:
- Ta sẽ lọc ra các dòng có rank=1 và last_week=1 (nghĩa là được top 1 liên tiếp)
- Sau đó groupby theo ID để đếm số tuần top 1 liên tiếp, nhưng phải cộng thêm 1 vì tuần đầu tiên được top 1 sẽ có last_week!=1
- Tiếp theo sẽ merge với dataframe songs (danh sách các bài hát có trong dataframe billboards) để lấy ra tên và ca sĩ trình bày bài hát
- Cuối cùng là sắp xếp các bài hát theo thứ tự giảm dần số tuần đạt top 1 liên tiếp trên BXH

In [18]:
df = new_billboards.query('rank==1 and last_week==1').groupby('ID')['date'].count() + 1

songs_consecutive_week_top1 = pd.DataFrame(data=df.values, index=df.index, columns=['weeks'])\
                                .reset_index()\
                                .merge(songs, how='inner', on='ID')\
                                .sort_values(by='weeks', ascending=False)\
                                .reset_index(drop=True)

songs_consecutive_week_top1

,ID,weeks,song,artist
0,1332,19,Old Town Road,Lil Nas X Featuring Billy Ray Cyrus
1,2432,16,Despacito,Luis Fonsi & Daddy Yankee Featuring Justin Bieber
2,10367,16,One Sweet Day,Mariah Carey & Boyz II Men
3,4758,14,I Will Always Love You,Whitney Houston
4,10153,14,Macarena (Bayside Boys Mix),Los Del Rio
...,...,...,...,...
704,14338,2,Everything She Wants,Wham!
705,14318,2,Everybody Wants To Rule The World,Tears For Fears
706,14312,2,Heaven,Bryan Adams
707,14297,2,A View To A Kill,Duran Duran
